In [25]:
import torch
import torch.nn as nn
import pandas as pd
import joblib
import time
import talib
import ccxt
import ta

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [26]:
# -------------------------
# CONFIG
# -------------------------
API_KEY = ""
API_SECRET = ""

symbol = "BTC/USDT"
model_timeframe = "15m"
price_timeframe = "1m"
limit = 200
window_size = 10

FEATURES = ['RSI', 'EMA12', 'EMA26', 'MACD', 'Signal', 'Histogram', 'DEMA9', 'SMA', 'TSI', '%K', '%D']

SAVED_MODEL = "greg_tech_5.pth"
SCALER_FILE = "scaler_15m.pkl"

min_tp_pct = 0.001
position_fraction = 1
check_interval_seconds = 1
simulate_only = True

atr_period = 14
atr_multiplier = 1.5  # Adjust ATR SL width


In [27]:
# -------------------------
# MODEL
# -------------------------
class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(CryptoLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

model = CryptoLSTM(input_dim=len(FEATURES), hidden_dim=128, num_layers=4, output_dim=3)
model.load_state_dict(torch.load(SAVED_MODEL))
model.to(device)
model.eval()

scaler = joblib.load(SCALER_FILE)


In [28]:
# -------------------------
# EXCHANGE
# -------------------------
exchange = ccxt.binance({'enableRateLimit': True})


In [29]:
def compute_tsi(close, r1=25, r2=13):
    delta = close.diff()
    ema1 = delta.ewm(span=r1, adjust=False).mean()
    ema2 = ema1.ewm(span=r2, adjust=False).mean()

    abs_delta = delta.abs()
    abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
    abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()

    tsi = 100 * (ema2 / abs_ema2)
    return tsi


In [30]:
import ta
def compute_indicators(df):
    out = {}
    out["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()
    out["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()
    out["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()
    out["MACD"] = out["EMA12"] - out["EMA26"]
    out["Signal"] = out["MACD"].ewm(span=9, adjust=False).mean()
    out["Histogram"] = out["MACD"] - out["Signal"]
    out["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)
    sma_window = 3
    out['SMA'] = ta.trend.sma_indicator(df['close'], window=sma_window)
    out['TSI'] = compute_tsi(df['close'])
    period = 14
    smooth_k = 3
    smooth_d = 3

    lowest_low = df["low"].rolling(period).min()
    highest_high = df["high"].rolling(period).max()

    out["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low)
    out["%K"] = out["%K"].rolling(smooth_k).mean()
    out["%D"] = out["%K"].rolling(smooth_d).mean()
    
    feat_df = pd.DataFrame(out, index=df.index)
    return feat_df


In [31]:
# -------------------------
# FETCH DATA
# -------------------------
def fetch_latest_ohlcv(symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.columns = [c.lower() for c in df.columns]
    return df


In [32]:
# -------------------------
# PREDICTION
# -------------------------
def predict_label_from_window(window_features):
    flat = scaler.transform(window_features)
    x = torch.tensor(flat, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(x)
        pred = torch.argmax(out, dim=1).item()
    return pred  # 0=down, 1=hold, 2=up


In [33]:
# -------------------------
# TRADING STATE
# -------------------------
balance = 1000.0
position = 0
entry_price = None
trade_size = 0.0
current_tp = None
current_sl = None
legs = 0

In [34]:
# -------------------------
# ATR-BASED FUNCTIONS
# -------------------------
def get_current_atr():
    df = fetch_latest_ohlcv(symbol, model_timeframe, atr_period + 1)

    # Previous close (shifted)
    df['prev_close'] = df['close'].shift(1)

    # True Range (TR)
    df['H-L'] = df['high'] - df['low']
    df['H-C'] = abs(df['high'] - df['prev_close'])
    df['L-C'] = abs(df['low'] - df['prev_close'])
    df['TR'] = df[['H-L', 'H-C', 'L-C']].max(axis=1)

    # Average True Range (ATR)
    df['ATR'] = df['TR'].rolling(atr_period).mean()

    return df['ATR'].iloc[-1]   # latest ATR value

def execute_buy(price):
    global balance, position, entry_price, trade_size, current_tp, current_sl, legs
    atr_now = get_current_atr()
    position = 1
    entry_price = price
    trade_size = balance * position_fraction
    current_sl = entry_price - (atr_now * atr_multiplier)
    current_tp = entry_price * (1 + min_tp_pct)
    legs = 0
    print(f"[ENTER LONG] price={price:.2f}, size={trade_size:.2f}, ATR_SL={current_sl:.2f}, TP={current_tp:.2f}")

def execute_sell_close(price):
    global balance, position, entry_price, trade_size, current_tp, current_sl, legs
    if position == 1 and entry_price is not None:
        pnl = trade_size * ((price - entry_price) / entry_price)
        balance += pnl
        print(f"[EXIT LONG] price={price:.2f}, P&L={pnl:.2f}, new_balance={balance:.2f}")
    position = 0
    entry_price = None
    current_tp = None
    current_sl = None
    legs = 0

def update_tp_sl(price):
    global entry_price, current_sl, current_tp, trade_size
    atr_now = get_current_atr()
    current_sl = max(current_sl, price - (atr_now * atr_multiplier)) if current_sl else price - (atr_now * atr_multiplier)
    current_tp = price * (1 + min_tp_pct)
    entry_price = price
    trade_size = balance * position_fraction
    print(f"[NEW LEG] new_entry={entry_price:.2f}, ATR_SL={current_sl:.2f}, TP={current_tp:.2f}")


In [ ]:
# -------------------------
# MAIN LOOP
# -------------------------
# Determine last reversal
back_df = fetch_latest_ohlcv(symbol, model_timeframe, 60 + window_size)
back_feat = compute_indicators(back_df).dropna()
preds = [predict_label_from_window(back_feat[FEATURES].values[i-window_size:i]) for i in range(window_size, len(back_feat))]

last_reversal = next((p for p in reversed(preds) if p != 1), None)

if last_reversal == 2:
    print("Last model prediction was UPTREND -> ENTER INITIAL POSITION")
    last_price = fetch_latest_ohlcv(symbol, price_timeframe, 2)['close'].values[-1]
    execute_buy(last_price)

elif last_reversal == 0:
    print("Last model prediction was DOWNTREND -> CONTINUE LONG AS FAKE REVERSAL")
    last_price = fetch_latest_ohlcv(symbol, price_timeframe, 2)['close'].values[-1]
    position = 1
    entry_price = last_price
    update_tp_sl(last_price)

print("Starting going-long logic with ATR-based SL + TP legs. Press Ctrl+C to stop.")
try:
    while True:
        df_ohlcv = fetch_latest_ohlcv(symbol, model_timeframe, limit)
        feat_df = compute_indicators(df_ohlcv).dropna()
        if len(feat_df) < window_size:
            print("Not enough data. Waiting...")
            time.sleep(10)
            continue

        last_rows = feat_df[FEATURES].values[-window_size:]
        pred = predict_label_from_window(last_rows)
        last_price = fetch_latest_ohlcv(symbol, price_timeframe, 2)['close'].values[-1]

        print(f"[{pd.Timestamp.utcnow()}] Predicted: {pred}, Price: {last_price:.2f}, Balance: {balance:.2f}, Position: {position}")

        # Enter long
        if pred == 2 and position == 0:
            execute_buy(last_price)

        if position == 1:
            # Stop loss
            if last_price <= current_sl:
                print("STOP LOSS HIT")
                execute_sell_close(last_price)
            # Take profit
            elif last_price >= current_tp:
                legs += 1
                pnl = trade_size * ((current_tp - entry_price) / entry_price)
                balance += pnl
                print(f"TAKE PROFIT REACHED (leg {legs}): realized {pnl:.2f}, balance now {balance:.2f}")
                update_tp_sl(current_tp)

            # Fake reversal handling
            if pred == 0:
                print("MODEL PREDICTS DOWNTREND -> Treating as next TP leg")
                update_tp_sl(last_price)
                if last_price >= current_tp:
                    legs += 1
                    pnl = trade_size * ((current_tp - entry_price) / entry_price)
                    balance += pnl
                    print(f"TAKE PROFIT REACHED (leg {legs}) on fake reversal: realized {pnl:.2f}, balance now {balance:.2f}")
                    update_tp_sl(current_tp)

        time.sleep(check_interval_seconds)

except KeyboardInterrupt:
    print("Stopped by user. Final balance:", balance)

Last model prediction was DOWNTREND -> CONTINUE LONG AS FAKE REVERSAL
[NEW LEG] new_entry=119186.33, ATR_SL=118936.34, TP=119305.52
Starting going-long logic with ATR-based SL + TP legs. Press Ctrl+C to stop.
[2025-10-02 12:39:13.536741+00:00] Predicted: 1, Price: 119170.33, Balance: 1000.00, Position: 1
[2025-10-02 12:39:15.278818+00:00] Predicted: 1, Price: 119190.08, Balance: 1000.00, Position: 1
[2025-10-02 12:39:17.005511+00:00] Predicted: 1, Price: 119200.13, Balance: 1000.00, Position: 1
[2025-10-02 12:39:18.715824+00:00] Predicted: 1, Price: 119200.13, Balance: 1000.00, Position: 1
[2025-10-02 12:39:20.427893+00:00] Predicted: 1, Price: 119200.12, Balance: 1000.00, Position: 1
[2025-10-02 12:39:22.207111+00:00] Predicted: 1, Price: 119208.58, Balance: 1000.00, Position: 1
[2025-10-02 12:39:23.937269+00:00] Predicted: 1, Price: 119208.58, Balance: 1000.00, Position: 1
[2025-10-02 12:39:25.676159+00:00] Predicted: 1, Price: 119208.58, Balance: 1000.00, Position: 1
[2025-10-02 12: